In [1]:
import warnings
warnings.filterwarnings('ignore')
from sqlalchemy import create_engine
import psycopg2
import psycopg2.extras
import config
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
import random

query = """select * from assets ORDER BY date ASC;"""
connection = psycopg2.connect(host=config.DB_HOST, database=config.DB_NAME, user= config.DB_USER, password=config.DB_PASS)
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
engine = create_engine(f'postgresql://{config.DB_USER}:{config.DB_PASS}@{config.DB_HOST}:{config.DB_PORT}/{config.DB_NAME}')

df = pd.read_sql_query(query,con=engine)
SYMBOLS = ['MSFT','AAPL','NVDA','UBER',]
IS_INVESTED = False

In [2]:
def get_rsi(df, rsi_period):
    chg = df['close'].diff(1)
    gain = chg.mask(chg<0,0)
    loss = chg.mask(chg>0,0)
    avg_gain = gain.ewm(com=rsi_period-1, min_periods=rsi_period).mean()
    avg_loss = loss.ewm(com=rsi_period-1, min_periods=rsi_period).mean()
    rs = abs(avg_gain/avg_loss)
    rsi = 100 - (100/(1+rs))
    return rsi

def add_vwap(df):
    '''Returns dataframe with additional columns:
        vwap (float): Volume Weighted Average Price
        vwap_var (float): % variance of close from vwap
    Args:
        df (pandas.DataFrame): Dataframe with at least columns:
            datetime
            open
            high
            low
            adj_close
            volume
    Returns:
        df (pandas.DataFrame)
    '''
    df['vwap'] = (df['volume']*(df['high']+df['low']+df['adjclose'])/3).cumsum()/df['volume'].cumsum()
    df['vwap'] = df['vwap'].fillna(df['adjclose'])
    df['vwap_var'] = (df['adjclose']/df['vwap'])-1
    return df

In [3]:
def add_indicators(df):
    # relative strength index
    df['rsi14'] = get_rsi(df, 14)
    # moving averages
    df['sma9'] = df['close'].rolling(9).mean()
    df['sma180'] = df['close'].rolling(180).mean()
    df['sma9_var'] = (df['close']/df['sma9'])-1
    df['sma180_var'] = (df['close']/df['sma180'])-1
    # spreads
    df['spread']=((df['close']/df['open'])-1).abs()
    df['spread14_e']=df['spread'].ewm(span=14).mean()
    # volume-based indicator
    df['volume14'] = df['volume'].rolling(14).mean()
    df['volume34'] = df['volume'].rolling(34).mean()
    df['volume14_34_var'] = (df['volume14']/df['volume34'])-1
    df = add_vwap(df)
    return df

In [4]:
df = add_indicators(df)

In [5]:
def drop_na_bf_fill(df):
    """drop_na_bf_fill"""
    df.dropna(axis=0,how='all',inplace=True)
    # df.dropna(axis=1,how='any',inplace=True)
    df.fillna(method='ffill', inplace=True)
    df.fillna(method='bfill', inplace=True)
    return df

In [6]:
def upside_down(df):
    symbols = df['name'].unique()
    df_msft = df[df['name'] == 'MSFT']
    symbol = 'MSFT'
    final_df = pd.DataFrame(data=df_msft[['close','rsi14','sma9','sma180','sma9_var','sma180_var','spread','spread14_e','volume14','volume34','volume14_34_var','vwap','vwap_var']].to_numpy(), index = df_msft['date'],columns=[[f'{symbol}_Close',f'{symbol}_Rsi14',f'{symbol}_Sma9',f'{symbol}_Sma180',f'{symbol}_Sma9_var',f'{symbol}_Sma180_var',f'{symbol}_Spread',f'{symbol}_Spread14_e',f'{symbol}_Volume14',f'{symbol}_Volume34',f'{symbol}_Volume14_34_var',f'{symbol}_Vwap',f'{symbol}_vwap_var']])
    for symbol in symbols:
        if symbol != 'MSFT':
            df_sym = df[df['name'] == symbol]
            df_tmp = pd.DataFrame(data=df_sym[['close','rsi14','sma9','sma180','sma9_var','sma180_var','spread','spread14_e','volume14','volume34','volume14_34_var','vwap','vwap_var']].to_numpy(), index = df_sym['date'],columns=[[f'{symbol}_Close',f'{symbol}_Rsi14',f'{symbol}_Sma9',f'{symbol}_Sma180',f'{symbol}_Sma9_var',f'{symbol}_Sma180_var',f'{symbol}_Spread',f'{symbol}_Spread14_e',f'{symbol}_Volume14',f'{symbol}_Volume34',f'{symbol}_Volume14_34_var',f'{symbol}_Vwap',f'{symbol}_vwap_var']])
            final_df =final_df.join(df_tmp)
    return drop_na_bf_fill(final_df)

In [7]:
df = upside_down(df)


In [8]:
def return_prev(df):
    for symbol in SYMBOLS:
        df[f'{symbol}_Prev_close']=df[f'{symbol}_Close'].shift(1)
        df[f'{symbol}_Return'] = np.array(df[f'{symbol}_Close']) / np.array(df[f'{symbol}_Prev_close']) - 1
        df[f'{symbol}_Log_Return'] = np.log(df[f'{symbol}_Return'] + 1)
    return df

def shifted_log_return(df):
    for symbol in SYMBOLS:
        df[f'{symbol}_Shifted_Log_Return'] = df[f'{symbol}_Log_Return'].shift(-1)
    return df


df = return_prev(df)
df = shifted_log_return(df)

In [13]:
MSFT_Shifted_Log_Return

,MSFT_Close,MSFT_Rsi14,MSFT_Sma9,MSFT_Sma180,MSFT_Sma9_var,MSFT_Sma180_var,MSFT_Spread,MSFT_Spread14_e,MSFT_Volume14,MSFT_Volume34,...,NVDA_Prev_close,NVDA_Return,NVDA_Log_Return,UBER_Prev_close,UBER_Return,UBER_Log_Return,MSFT_Shifted_Log_Return,AAPL_Shifted_Log_Return,NVDA_Shifted_Log_Return,UBER_Shifted_Log_Return
date,,,,,,,,,,,,,,,,,,,,,
2022-03-22 13:30:00,300.612213,48.766782,464.715001,625.734044,-0.352205,-0.514698,0.002709,0.002709,6.658724e+05,6.351245e+05,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000325,0.003697,0.002639,0.003661
2022-03-22 13:35:00,300.709991,48.766782,464.715001,625.734044,-0.352205,-0.514698,0.000366,0.005896,6.658724e+05,6.351245e+05,...,266.950012,0.002642,0.002639,32.391201,0.003668,0.003661,0.001097,0.000265,-0.003764,-0.008790
2022-03-22 13:40:00,301.040100,48.766782,464.715001,625.734044,-0.352205,-0.514698,0.000965,0.003999,6.658724e+05,6.351245e+05,...,267.655396,-0.003757,-0.003764,32.509998,-0.008751,-0.008790,-0.003561,-0.002578,-0.000600,-0.004525
2022-03-22 13:45:00,299.970001,48.760684,479.530001,625.734044,-0.374450,-0.514698,0.003621,0.003945,6.788922e+05,6.351245e+05,...,266.649902,-0.000600,-0.000600,32.225498,-0.004515,-0.004525,-0.001903,-0.001444,-0.000901,0.014391
2022-03-22 13:50:00,299.399689,48.661779,768.270079,625.734044,-0.610294,-0.514698,0.001801,0.002519,6.065889e+05,6.351245e+05,...,266.489990,-0.000901,-0.000901,32.080002,0.014495,0.014391,0.001703,0.000782,0.001576,0.006279
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-09 19:35:00,264.910004,48.985945,370.491942,496.751940,-0.284978,-0.466716,0.000000,0.002467,6.456458e+05,5.205829e+05,...,169.702499,0.000280,0.000280,23.184999,0.000000,0.000000,-0.002495,-0.003349,-0.001769,-0.007142
2022-05-09 19:40:00,264.250000,49.229044,367.598323,484.142657,-0.281145,-0.454190,0.002454,0.002454,7.184594e+05,6.045602e+05,...,169.750000,-0.001767,-0.001769,23.184999,-0.007117,-0.007142,-0.001666,-0.000987,-0.003677,-0.004354
2022-05-09 19:45:00,263.809998,49.094719,377.631668,496.377695,-0.301409,-0.468530,0.001665,0.002318,8.040694e+05,6.607576e+05,...,169.449997,-0.003671,-0.003677,23.020000,-0.004344,-0.004354,0.000644,-0.001656,0.001018,-0.001310


In [ ]:
import numpy as np
import pandas as pd
from gym.utils import seeding
import gym
from gym import spaces
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import pickle

# shares normalization factor
# 100 shares per trade
HMAX_NORMALIZE = 100
# initial amount of money we have in our account
INITIAL_ACCOUNT_BALANCE=1000000
# total number of stocks in our portfolio
STOCK_DIM = 30
# transaction fee: 1/1000 reasonable percentage
TRANSACTION_FEE_PERCENT = 0.001

# turbulence index: 90-150 reasonable threshold
#TURBULENCE_THRESHOLD = 140
REWARD_SCALING = 1e-4

class StockEnvTrade(gym.Env):
    """A stock trading environment for OpenAI gym"""
    metadata = {'render.modes': ['human']}

    def __init__(self, df,day = 0,turbulence_threshold=140
                 ,initial=True, previous_state=[], model_name='', iteration=''):
        #super(StockEnv, self).__init__()
        #money = 10 , scope = 1
        self.day = day
        self.df = df
        self.initial = initial
        self.previous_state = previous_state
        # action_space normalization and shape is STOCK_DIM
        self.action_space = spaces.Box(low = -1, high = 1,shape = (STOCK_DIM,)) 
        # Shape = 181: [Current Balance]+[prices 1-30]+[owned shares 1-30] 
        # +[macd 1-30]+ [rsi 1-30] + [cci 1-30] + [adx 1-30]
        self.observation_space = spaces.Box(low=0, high=np.inf, shape = (181,))
        # load data from a pandas dataframe
        self.data = self.df.loc[self.day,:]
        self.terminal = False     
        self.turbulence_threshold = turbulence_threshold
        # initalize state
        self.state = [INITIAL_ACCOUNT_BALANCE] + \
                      self.data.MSFT_Close.values.tolist() + \
                      [0]*STOCK_DIM
        # initialize reward
        self.reward = 0
        self.turbulence = 0
        self.cost = 0
        self.trades = 0
        # memorize all the total balance change
        self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
        self.rewards_memory = []
        #self.reset()
        self._seed()
        self.model_name=model_name        
        self.iteration=iteration


    def _sell_stock(self, index, action):
        # perform sell action based on the sign of the action
        if self.turbulence<self.turbulence_threshold:
            if self.state[index+STOCK_DIM+1] > 0:
                #update balance
                self.state[0] += \
                self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
                 (1- TRANSACTION_FEE_PERCENT)
                
                self.state[index+STOCK_DIM+1] -= min(abs(action), self.state[index+STOCK_DIM+1])
                self.cost +=self.state[index+1]*min(abs(action),self.state[index+STOCK_DIM+1]) * \
                 TRANSACTION_FEE_PERCENT
                self.trades+=1
            else:
                pass
        else:
            # if turbulence goes over threshold, just clear out all positions 
            if self.state[index+STOCK_DIM+1] > 0:
                #update balance
                self.state[0] += self.state[index+1]*self.state[index+STOCK_DIM+1]* \
                              (1- TRANSACTION_FEE_PERCENT)
                self.state[index+STOCK_DIM+1] =0
                self.cost += self.state[index+1]*self.state[index+STOCK_DIM+1]* \
                              TRANSACTION_FEE_PERCENT
                self.trades+=1
            else:
                pass
    
    def _buy_stock(self, index, action):
        # perform buy action based on the sign of the action
        if self.turbulence< self.turbulence_threshold:
            available_amount = self.state[0] // self.state[index+1]
            # print('available_amount:{}'.format(available_amount))
            
            #update balance
            self.state[0] -= self.state[index+1]*min(available_amount, action)* \
                              (1+ TRANSACTION_FEE_PERCENT)

            self.state[index+STOCK_DIM+1] += min(available_amount, action)
            
            self.cost+=self.state[index+1]*min(available_amount, action)* \
                              TRANSACTION_FEE_PERCENT
            self.trades+=1
        else:
            # if turbulence goes over threshold, just stop buying
            pass
        
    def step(self, actions):
        # print(self.day)
        self.terminal = self.day >= len(self.df.index.unique())-1
        # print(actions)

        if self.terminal:
            plt.plot(self.asset_memory,'r')
            plt.savefig('results/account_value_trade_{}_{}.png'.format(self.model_name, self.iteration))
            plt.close()
            df_total_value = pd.DataFrame(self.asset_memory)
            df_total_value.to_csv('results/account_value_trade_{}_{}.csv'.format(self.model_name, self.iteration))
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            print("previous_total_asset:{}".format(self.asset_memory[0]))           

            print("end_total_asset:{}".format(end_total_asset))
            print("total_reward:{}".format(self.state[0]+sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))- self.asset_memory[0] ))
            print("total_cost: ", self.cost)
            print("total trades: ", self.trades)

            df_total_value.columns = ['account_value']
            df_total_value['daily_return']=df_total_value.pct_change(1)
            sharpe = (4**0.5)*df_total_value['daily_return'].mean()/ \
                  df_total_value['daily_return'].std()
            print("Sharpe: ",sharpe)
            
            df_rewards = pd.DataFrame(self.rewards_memory)
            df_rewards.to_csv('results/account_rewards_trade_{}_{}.csv'.format(self.model_name, self.iteration))
            
            # print('total asset: {}'.format(self.state[0]+ sum(np.array(self.state[1:29])*np.array(self.state[29:]))))
            #with open('obs.pkl', 'wb') as f:  
            #    pickle.dump(self.state, f)
            
            return self.state, self.reward, self.terminal,{}

        else:
            # print(np.array(self.state[1:29]))

            actions = actions * HMAX_NORMALIZE
            #actions = (actions.astype(int))
            if self.turbulence>=self.turbulence_threshold:
                actions=np.array([-HMAX_NORMALIZE]*STOCK_DIM)
                
            begin_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            #print("begin_total_asset:{}".format(begin_total_asset))
            
            argsort_actions = np.argsort(actions)
            
            sell_index = argsort_actions[:np.where(actions < 0)[0].shape[0]]
            buy_index = argsort_actions[::-1][:np.where(actions > 0)[0].shape[0]]

            for index in sell_index:
                # print('take sell action'.format(actions[index]))
                self._sell_stock(index, actions[index])

            for index in buy_index:
                # print('take buy action: {}'.format(actions[index]))
                self._buy_stock(index, actions[index])

            self.day += 1
            self.data = self.df.loc[self.day,:]         
            self.turbulence = self.data['turbulence'].values[0]
            #print(self.turbulence)
            #load next state
            # print("stock_shares:{}".format(self.state[29:]))
            self.state =  [self.state[0]] + \
                    self.data.adjcp.values.tolist() + \
                    list(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]) + \
                    self.data.macd.values.tolist() + \
                    self.data.rsi.values.tolist() + \
                    self.data.cci.values.tolist() + \
                    self.data.adx.values.tolist()
            
            end_total_asset = self.state[0]+ \
            sum(np.array(self.state[1:(STOCK_DIM+1)])*np.array(self.state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            self.asset_memory.append(end_total_asset)
            #print("end_total_asset:{}".format(end_total_asset))
            
            self.reward = end_total_asset - begin_total_asset            
            # print("step_reward:{}".format(self.reward))
            self.rewards_memory.append(self.reward)
            
            self.reward = self.reward*REWARD_SCALING


        return self.state, self.reward, self.terminal, {}

    def reset(self):  
        if self.initial:
            self.asset_memory = [INITIAL_ACCOUNT_BALANCE]
            self.day = 0
            self.data = self.df.loc[self.day,:]
            self.turbulence = 0
            self.cost = 0
            self.trades = 0
            self.terminal = False 
            #self.iteration=self.iteration
            self.rewards_memory = []
            #initiate state
            self.state = [INITIAL_ACCOUNT_BALANCE] + \
                          self.data.adjcp.values.tolist() + \
                          [0]*STOCK_DIM + \
                          self.data.macd.values.tolist() + \
                          self.data.rsi.values.tolist()  + \
                          self.data.cci.values.tolist()  + \
                          self.data.adx.values.tolist() 
        else:
            previous_total_asset = self.previous_state[0]+ \
            sum(np.array(self.previous_state[1:(STOCK_DIM+1)])*np.array(self.previous_state[(STOCK_DIM+1):(STOCK_DIM*2+1)]))
            self.asset_memory = [previous_total_asset]
            #self.asset_memory = [self.previous_state[0]]
            self.day = 0
            self.data = self.df.loc[self.day,:]
            self.turbulence = 0
            self.cost = 0
            self.trades = 0
            self.terminal = False 
            #self.iteration=iteration
            self.rewards_memory = []
            #initiate state
            #self.previous_state[(STOCK_DIM+1):(STOCK_DIM*2+1)]
            #[0]*STOCK_DIM + \

            self.state = [ self.previous_state[0]] + \
                          self.data.adjcp.values.tolist() + \
                          self.previous_state[(STOCK_DIM+1):(STOCK_DIM*2+1)]+ \
                          self.data.macd.values.tolist() + \
                          self.data.rsi.values.tolist()  + \
                          self.data.cci.values.tolist()  + \
                          self.data.adx.values.tolist() 
            
        return self.state
    
    def render(self, mode='human',close=False):
        return self.state
    

    def _seed(self, seed=None):
        self.np_random, seed = seeding.np_random(seed)
        return [seed]